In [15]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [16]:
import numpy as np
import pandas as pd
import random
import os
#from imutils import paths
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

import autokeras as ak
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, ReLU, Dropout
from tensorflow.keras.models import Model

In [17]:
#declear path to your data
drebin_data_path = 'data\drebin.csv'
columns = list(pd.read_csv('dataset-features-categories.csv', header = None).iloc[:,0])
# Importing the dataset
Drebin_data = pd.read_csv(drebin_data_path, names = columns)

X = Drebin_data.iloc[:,range(0,Drebin_data.shape[1]-1)].values
y = Drebin_data.iloc[:, -1].values

lb = LabelBinarizer()
y = lb.fit_transform(y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size = 0.3, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


# Train-Test

In [4]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv_method = RepeatedStratifiedKFold(n_splits=5,  n_repeats=3, random_state=999)
from sklearn.preprocessing import PowerTransformer

"""params_NB = {'var_smoothing': np.logspace(-10,10, num=2000)}

gs_NB = GridSearchCV(estimator=GaussianNB(), param_grid=params_NB, cv=cv_method,verbose=1,scoring='accuracy')
"""
# "GaussianNB": gs_NB, "RFC": RandomForestClassifier(), "DT": DecisionTreeClassifier(), "SGD": SGDClassifier(),
# 79.4%, 98.8%, 97.3%, 97.6%,  
classifiers = {"SVM": SVC(kernel = 'linear', degree=3), "MLP": MLPClassifier(random_state=1, max_iter=300), "XGB": XGBClassifier(), "LGBM": LGBMClassifier()} 

for classifier_pair in classifiers.items():
    print("---------------------------")
    print(classifier_pair[0])
    
    classifier = classifier_pair[1]
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix', cm)

    #compute accuracy_score
    accuracy = acc(y_test, y_pred)
    print('accuracy', accuracy)

    #compute precision score
    precision_score = precision(y_test, y_pred, average='micro')
    print('precision', precision_score)

    #compute recall score
    recall_score = recall(y_test, y_pred)
    print('recall', recall_score)

    #compute f1 score
    f1_score = f1(y_test, y_pred)
    print('f1', f1_score)
    
print("---------------------------")

---------------------------
SVM
Confusion Matrix [[2851   47]
 [  45 1568]]
accuracy 0.9796054090002216
precision 0.9796054090002216
recall 0.972101673899566
f1 0.9714993804213135
---------------------------
MLP
Confusion Matrix [[2880   18]
 [  29 1584]]
accuracy 0.9895810241631567
precision 0.9895810241631567
recall 0.9820210787352759
f1 0.9853810264385693
---------------------------
XGB
Confusion Matrix [[2882   16]
 [  27 1586]]
accuracy 0.9904677455109732
precision 0.9904677455109732
recall 0.9832610043397396
f1 0.9866251944012442
---------------------------
LGBM
Confusion Matrix [[2884   14]
 [  29 1584]]
accuracy 0.9904677455109732
precision 0.9904677455109732
recall 0.9820210787352759
f1 0.9866085331672375
---------------------------


In [5]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=10
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    x=X_train,
    y=y_train,
    epochs=15,
)

# Evaluate the best model with testing data.
print(clf.evaluate(x=X_test, y=y_test))

Trial 10 Complete [00h 01m 51s]
val_accuracy: 0.9821858406066895

Best val_accuracy So Far: 0.9879634380340576
Total elapsed time: 00h 17m 00s
INFO:tensorflow:Oracle triggered exit
Epoch 1/15
329/329 [==============================] - 6s 8ms/step - loss: 0.1045 - accuracy: 0.9627
Epoch 2/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0432 - accuracy: 0.9860
Epoch 3/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0215 - accuracy: 0.9948
Epoch 4/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0124 - accuracy: 0.9967
Epoch 5/15
329/329 [==============================] - 2s 8ms/step - loss: 0.0166 - accuracy: 0.9952
Epoch 6/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0204 - accuracy: 0.9929
Epoch 7/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0115 - accuracy: 0.9966
Epoch 8/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0087 - accuracy: 0.9975
Epoch 9/15
329/329 

In [6]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 215)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 215)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 215)              431       
 n)                                                              
                                                                 
 dense (Dense)               (None, 512)               110592    
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                             

In [18]:
def autoKerasModel(path_best_model):
    saved_model = keras.models.load_model(path_best_model, compile=True)
    input_layer = Input(shape=(215,))
    x = saved_model.layers[1](input_layer)
    x = saved_model.layers[2](x)
    x = Dense(units=32)(x)
    x = ReLU()(x)
    x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Dense(units=32)(x)
    x = ReLU()(x)
    x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = tf.keras.layers.Dropout(rate=0.5)(x)
    x = Dense(units=1)(x)
    x = saved_model.layers[-1](x)
    new_model = Model(inputs=input_layer, outputs=x)
    return new_model

In [19]:
new_model=autoKerasModel('best_model')
new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
new_model.fit(X_train, y_train, epochs=15)
results = new_model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

Epoch 1/15
329/329 [==============================] - 5s 7ms/step - loss: 0.5899 - accuracy: 0.7323
Epoch 2/15
329/329 [==============================] - 2s 7ms/step - loss: 0.2544 - accuracy: 0.9015
Epoch 3/15
329/329 [==============================] - 2s 7ms/step - loss: 0.1641 - accuracy: 0.9405
Epoch 4/15
329/329 [==============================] - 2s 7ms/step - loss: 0.1412 - accuracy: 0.9520
Epoch 5/15
329/329 [==============================] - 2s 7ms/step - loss: 0.1210 - accuracy: 0.9616
Epoch 6/15
329/329 [==============================] - 2s 7ms/step - loss: 0.1030 - accuracy: 0.9663
Epoch 7/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0999 - accuracy: 0.9681
Epoch 8/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0907 - accuracy: 0.9705
Epoch 9/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0856 - accuracy: 0.9738
Epoch 10/15
329/329 [==============================] - 2s 7ms/step - loss: 0.0751 - accuracy: 0.9743

In [10]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 215)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 215)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 215)              431       
 n)                                                              
                                                                 
 dense_3 (Dense)             (None, 32)                6912      
                                                                 
 re_lu_2 (ReLU)              (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0   

In [11]:
a=keras.models.load_model('best_model', compile=True)
print(a.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 215)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 215)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 215)              431       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                6912      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0     